This problem was asked by Microsoft.

Given a dictionary of words and a string made up of those words (no spaces), return the original sentence in a list. If there is more than one possible reconstruction, return any of them. If there is no possible reconstruction, then return null.

For example, given the set of words 'quick', 'brown', 'the', 'fox', and the string "thequickbrownfox", you should return ['the', 'quick', 'brown', 'fox'].

Given the set of words 'bed', 'bath', 'bedbath', 'and', 'beyond', and the string "bedbathandbeyond", return either ['bed', 'bath', 'and', 'beyond] or ['bedbath', 'and', 'beyond'].

# Answer

In [5]:
# O(lst)
def word_separator(string, lst):
    lst_ordered = []
    str_trimmed = string
    for i in range(len(string)):
        for word in lst:
            if word == str_trimmed[:len(word)]:
                lst_ordered.append(word)
                str_trimmed = str_trimmed[len(word):]
                break
    if len(lst_ordered) == 0:
        return None
    else: 
        return lst_ordered
            

In [8]:
word_separator('bedbathandbeyond', ['bed', 'bath', 'bedbath', 'beyond', 'and'])

['bed', 'bath', 'and', 'beyond']

# Solution

In [4]:
test = 'abc'
test1 = test
test1[:2]

'ab'